In [1]:
import numpy as np
import pandas as pd
import twosigmafunc

In [2]:
train = pd.read_hdf('train.h5')

In [3]:
train.sort_values(['id', 'timestamp'], inplace=True)
train.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
131062,0,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007108
131895,0,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001950
132728,0,169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017724
133561,0,170,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.727659,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,0.012934
134393,0,171,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.727659,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,-0.025229


In [4]:
origin_features = twosigmafunc.origin_features(train)
nan_features = twosigmafunc.addtional_features(origin_features, '_nan')
train['id_diff'] = train.id.diff()
initial_stage = train.loc[train.id_diff!=0]

In [5]:
initial_stage['nan_sum'] = initial_stage.isnull().sum(axis=1)

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
initial_stage.nan_sum.unique()

array([107, 106,  24,  22,  28,  38,  21,  30,  60,  27,  42,  51,  54,
        33,  26,  36,  35,  29,  23,  25,  20,  37,  47,  61,  43,  50,
        46,  34,  40,  31,  39,  32,  44,  63,  64,  82,  88,  56,  41,
        81,  59,  55,  52,  57,  89,  48,  58,  68,  49,  62,  66,  45,
        69,  65,  80])

In [7]:
for col, nan in zip(origin_features, nan_features):
    initial_stage[nan] = initial_stage[col].isnull()

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [9]:
# del train
inert = []
for n in range(2,10):
    km = KMeans(n_clusters=n, n_jobs=1, random_state=11, n_init=10)
    km.fit(initial_stage[nan_features + ['nan_sum']])
    score = silhouette_score(initial_stage[nan_features + ['nan_sum']], km.labels_)
    inert.append((n,score))

In [10]:
inert

[(2, 0.86891646043095638),
 (3, 0.82780727961783229),
 (4, 0.78999489664043965),
 (5, 0.77572947077190957),
 (6, 0.78254519477431117),
 (7, 0.77240612987586688),
 (8, 0.76355227039873197),
 (9, 0.7658268251075232)]

In [9]:
km = KMeans(n_clusters=5, n_jobs=1, random_state=11, n_init=20)
km.fit(initial_stage[nan_features + ['nan_sum']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=20, n_jobs=1, precompute_distances='auto',
    random_state=11, tol=0.0001, verbose=0)

In [10]:
from collections import defaultdict
group_id = defaultdict(list)
for id_, group in zip(initial_stage.id, km.labels_):
    group_id[group].append(id_)

In [11]:
for key, gp in group_id.items():
    print(len(gp))

188
680
166
250
140


In [53]:
def label(id):
    if id in group_id[0]:
        return 'a'
    elif id in group_id[1]:
        return 'b'
    elif id in group_id[2]:
        return 'c'
    elif id in group_id[3]:
        return 'd'
    elif id in group_id[4]:
        return 'e'
    elif id in group_id[5]:
        return 'f'
    elif id in group_id[6]:
        return 'g'
    elif id in group_id[7]:
        return 'h'
    elif id in group_id[8]:
        return 'i'
    else:
        return 'n'

In [13]:
train.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y,id_diff
131062,0,167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007108,NaN
131895,0,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001950,0.0
132728,0,169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017724,0.0
133561,0,170,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,0.012934,0.0
134393,0,171,-0.230583,0.488096,0.93592,0.028222,-0.083071,-0.240929,NaN,0.212425,...,0.0,0.0,0.0,-0.160478,NaN,0.0,0.0,NaN,-0.025229,0.0


In [14]:
label = [label(id) for id in train.id]

In [50]:
set(label)

{'a', 'b', 'c', 'd', 'e'}

In [15]:
label_dummy = pd.get_dummies(label, prefix='Label')
train.drop(['id', 'id_diff'], axis=1, inplace=True)
train = pd.concat([train, label_dummy], axis=1)
median = train.median()

In [16]:
for col, nan in zip(origin_features, nan_features):
    train[nan] = train[col].isnull()
    median[nan] = 0

In [17]:
train = train.fillna(median)

In [18]:
train.head()

,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,fundamental_3,...,technical_35_nan,technical_36_nan,technical_37_nan,technical_38_nan,technical_39_nan,technical_40_nan,technical_41_nan,technical_42_nan,technical_43_nan,technical_44_nan
0,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,-0.179404,...,False,False,True,True,True,False,True,True,False,True
1,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,0.216281,...,False,False,True,True,True,False,True,True,False,True
2,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,-0.040183,...,False,False,True,True,True,False,True,True,False,True
3,0,-0.000837,0.005523,0.021095,0.002476,0.011752,0.178495,-0.007395,-0.007262,-0.097903,...,False,False,True,True,True,False,True,True,False,True
4,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,0.196391,...,False,False,True,True,True,False,True,True,False,True


In [19]:
from sklearn.ensemble import ExtraTreesRegressor
etr =  ExtraTreesRegressor(n_estimators=60, n_jobs=-1,max_depth=4, random_state=17, verbose=0)

In [20]:
X_train, y_train, X_test, y_test = twosigmafunc.split_data(train, origin_features + nan_features)

In [21]:
etr.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=4,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=60, n_jobs=-1, oob_score=False, random_state=17,
          verbose=0, warm_start=False)

In [23]:
y_pred = etr.predict(X_test)
twosigmafunc.R_score(y_pred, y_test)

0.021350026493180539

# add clusters 

In [27]:
X_train, y_train, X_test, y_test = twosigmafunc.split_data(train, origin_features + nan_features + ['Label_a', 'Label_b', 'Label_c', 'Label_d', 'Label_e'])

In [29]:
etr.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=4,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_split=1e-07, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=60, n_jobs=-1, oob_score=False, random_state=17,
          verbose=0, warm_start=False)

In [30]:
y_pred = etr.predict(X_test)
twosigmafunc.R_score(y_pred, y_test)

-0.0069139774386441271

In [31]:
imp = pd.DataFrame({'feature':X_train.columns, 'important':etr.feature_importances_})
imp.sort_values('important', ascending=False)

,feature,important
85,technical_20,0.134829
93,technical_30,0.121247
217,Label_b,0.111148
218,Label_c,0.104304
220,Label_e,0.057408
15,fundamental_11,0.042117
74,technical_7,0.036381
196,technical_24_nan,0.030301
103,technical_40,0.024341
86,technical_21,0.020578
